# Citizen Science Notebook - TAP Tutorial
This notebook demonstrates the usage of using the TAP query service to curate tabular data and send it to Zooniverse

## Create a Zooniverse Account
If you haven't already, [create a Zooniverse account here.](https://www.zooniverse.org/)
After creating your account, return to this notebook.

## Terminal Prep Work
The follow cell will run the necessary terminal commands that make this notebook possible.

In [1]:
# Install panoptes client package to dependencies
!yum install zip
# !python -m pip install panoptes-client
!mkdir -p project/citizen-science/astro-cutouts/
!mkdir -p project/citizen-science/org
!python -m pip install google-cloud-storage

Loaded plugins: fastestmirror, ovl
ovl: Error while doing RPMdb copy-up:
[Errno 13] Permission denied: '/var/lib/rpm/.dbenv.lock'
You need to be root to perform this command.
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# temp workaround code
!python -m pip install -U git+https://github.com/zooniverse/panoptes-python-client.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/zooniverse/panoptes-python-client.git to /tmp/pip-req-build-v2c2sorn
  Running command git clone --filter=blob:none --quiet https://github.com/zooniverse/panoptes-python-client.git /tmp/pip-req-build-v2c2sorn
  Resolved https://github.com/zooniverse/panoptes-python-client.git to commit da059f5e1792569393f991faf71316e902e7f95b
  Preparing metadata (setup.py) ... done


## Log in to Zooniverse
Now that you have a Zooniverse account, log into the Zooniverse(Panoptes) client.

In [4]:
# Log into Zooniverse
import panoptes_client
client = panoptes_client.Panoptes.connect(login="interactive")

Enter your Zooniverse credentials...


Username:  ericrosas
 ···········


 ## Look Up Your Zooniverse Project
 
 IMPORTANT: Your Zooniverse project must be set to "public", a "private" project will not work.
 
 The following code will not work if you have not authenticated in the cell titled "Log in to Zooniverse". </br>
 Supply the project name in the variable below.
 </br></br>
 Not that the `Project.find()` method expects the project name to reflect the "slug" of your project, if you don't know what a "slug" is in this context, see:</br>
 https://www.zooniverse.org/talk/18/967061?comment=1898157&page=1

In [5]:
from panoptes_client import Project, SubjectSet

## TO-DO: Enter your email address and the slug of your project
email = "erosas@lsst.org"
slugName = "ericrosas/test-project" # Add your slug name

project = Project.find(slug=slugName)

projectId = project.id

print(projectId)

print(project.display_name)

for sub in project.links.subject_sets:
    print(sub.completeness)
print("IMPORTANT! Scroll down to the cell that starts with a title of 'IMPORTANT! Run the below cell...' before proceeding to the next cell")

18676
Test Project
{}
IMPORTANT! Scroll down to the cell that starts with a title of 'IMPORTANT! Run the below cell...' before proceeding to the next cell


In [ ]:
# import numpy as np
# import threading

# urls = []

# def test_func(cutout):
#     print("in test_func")
#     print(cutout)
#     urls.append(...cutout)

# arr = x = np.arange(105)
# arrs = np.split(arr, len(arr) / 10)
# for cutout in arrs:
#     t = threading.Thread(target=test_func, args=(cutout,))
#     t.start()
#     t.join()
    
# print("done!")
# print(urls)
    

## Create Image Cutout from HiPS Server
You can create a cutout image of a HiPS survey/server by doing the following:

In [22]:
# Load up the TAP query service
from lsst.rsp import get_tap_service, retrieve_query
service = get_tap_service()

def create_csv_from_tabular_data(tab_data):
    tap_as_csv = ''
    for i, col in enumerate(tab_data.fieldnames):
        if i > 0:
            tap_as_csv += ','
        tap_as_csv += col

    tap_as_csv += "\n"

    for result in object_results:
        for i, col in enumerate(result.values()):
            if i > 0:
                tap_as_csv += ','
            tap_as_csv += str(col)
        tap_as_csv += "\n"
    return tap_as_csv


### Object table query
object_results = service.search("SELECT objectId, coord_dec, Coord_ra, g_ra, i_ra, r_ra, u_ra, y_ra, z_ra, g_decl, i_decl, r_decl, u_decl, y_decl, z_decl, g_bdFluxB, i_bdFluxB, r_bdFluxB, u_bdFluxB, y_bdFluxB, z_bdFluxB, g_bdFluxD, i_bdFluxD, r_bdFluxD, u_bdFluxD, y_bdFluxD, z_bdFluxD, g_bdReB, i_bdReB, r_bdReB, u_bdReB, y_bdReB, z_bdReB, g_bdReD, i_bdReD, r_bdReD, u_bdReD, y_bdReD, z_bdReD "\
                         "FROM dp02_dc2_catalogs.Object", maxrec=10)
# object_results_tab = object_results.to_table()
# object_results_tab
object_results_csv = create_csv_from_tabular_data(object_results)

### DiaObject table query
dia_object_results = service.search("SELECT decl, ra, gPSFluxChi2, iPSFluxChi2, rPSFluxChi2, uPSFluxChi2, yPSFluxChi2, zPSFluxChi2, gPSFluxMax, iPSFluxMax, rPSFluxMax, uPSFluxMax, yPSFluxMax, zPSFluxMax, gPSFluxMin, iPSFluxMin, rPSFluxMin, uPSFluxMin, yPSFluxMin, zPSFluxMin, gPSFluxMean, iPSFluxMean, rPSFluxMean, uPSFluxMean, yPSFluxMean, zPSFluxMean, gPSFluxNdata, iPSFluxNdata, rPSFluxNdata, uPSFluxNdata, yPSFluxNdata, zPSFluxNdata "\
                         "FROM dp02_dc2_catalogs.DiaObject", maxrec=10)
# dia_object_results_tab = dia_object_results.to_table()
# dia_object_results_tab
dia_object_results_csv = create_csv_from_tabular_data(dia_object_results)

### ForcedSource table query
forced_source_results = service.search("SELECT forcedSourceId, objectId, parentObjectId, coord_ra, coord_dec, skymap, tract, patch, band, ccdVisitId, detect_isPatchInner, detect_isPrimary, detect_isTractInner,localBackground_instFluxErr, localBackground_instFlux, localPhotoCalibErr, localPhotoCalib_flag, localPhotoCalib, localWcs_CDMatrix_1_1, localWcs_CDMatrix_1_2, localWcs_CDMatrix_2_1, localWcs_CDMatrix_2_2, localWcs_flag, pixelFlags_bad, pixelFlags_crCenter, pixelFlags_cr, pixelFlags_edge, pixelFlags_interpolatedCenter, pixelFlags_interpolated, pixelFlags_saturatedCenter, pixelFlags_saturated, pixelFlags_suspectCenter, pixelFlags_suspect, psfDiffFluxErr, psfDiffFlux_flag, psfDiffFlux, psfFluxErr, psfFlux_flag, psfFlux "\
                         "FROM dp02_dc2_catalogs.ForcedSource", maxrec=10)
# forced_source_results_tab = forced_source_results.to_table()
# forced_source_results_tab
forced_source_results_csv = create_csv_from_tabular_data(forced_source_results)

print("Object results:")
print(object_results_csv)

print("DiaObject results:")
print(dia_object_results_csv)

print("ForcedSource results:");
print(forced_source_results_csv)

Object results:
objectId,coord_dec,Coord_ra,g_ra,i_ra,r_ra,u_ra,y_ra,z_ra,g_decl,i_decl,r_decl,u_decl,y_decl,z_decl,g_bdFluxB,i_bdFluxB,r_bdFluxB,u_bdFluxB,y_bdFluxB,z_bdFluxB,g_bdFluxD,i_bdFluxD,r_bdFluxD,u_bdFluxD,y_bdFluxD,z_bdFluxD,g_bdReB,i_bdReB,r_bdReB,u_bdReB,y_bdReB,z_bdReB,g_bdReD,i_bdReD,r_bdReD,u_bdReD,y_bdReD,z_bdReD
1329503072027709206,-42.1511266,69.5575035,69.5574987,69.5575035,69.5575033,69.5575026,69.5575025,69.5575034,-42.1511256,-42.1511266,-42.1511264,-42.1511265,-42.1511251,-42.1511265,372.2873023,531.9323122,428.9036831,262.4370532,882.0089006,803.9951925,322.7598806,481.0387995,365.5218903,235.9502341,794.8639895,717.5093349,2.1598829,1.9409479,2.2703853,1.9061983,1.8471988,1.9592959,1.9194363,1.9011721,1.9428003,1.8780927,1.8578926,1.8832815
1329503072027709207,-42.150509,69.5574221,69.5574205,69.5574221,69.5574214,69.5574197,69.557415,69.5574214,-42.1505081,-42.150509,-42.1505086,-42.1505077,-42.1505044,-42.1505085,164.4884664,389.9938466,203.1791396,166.70049

## Create a new subject set
Run this before running the "Send Data" cell.

In [ ]:
subject_set_name = "app engine test - multithreading 22" # give your subject set a name
subject_set_name

## Send the cutouts to Zooniverse
Don't click the below cell multiple times, the upload will fail if multiple runs are attempted.

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
h = display(display_id='my-display')
h.display(None)

__cit_sci_data_type = _HIPS_CUTOUTS # Important: DO NOT change this value
send_data(subject_set_name, cutouts)

## Show additional messages
After running the above cell and receiving the message that the transfer has completed, run the below cell to show additional messages that were accrued during processing.

In [ ]:
print(edc_response["messages"])

## Explicitly check the status of your data batch
Is the send_data() call above stalling on "Notifying the Rubin EPO Data Center..." step? Run the below cell every few minutes to check the status of your data. Large datasets can cause the response to get lost, but that does not necessarily mean that your data was not sent to Zooniverse.

In [ ]:
res = check_status()
print(res["status"])
print(res["manifest_url"])
print(res["messages"])
if res["status"] == "success":
    global manifest_url
    manifest_url = res["manifest_url"]
    send_zooniverse_manifest()

## IMPORTANT! Run the below cell to activate the Rubin Citizen Science SDK

In [ ]:
# HiPS astrocutout libraries
from astroquery.hips2fits import hips2fits
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import astropy.units as u
from astropy.coordinates import Longitude, Latitude, Angle

# Zooniverse libraries
# from panoptes_client import Panoptes, Project, SubjectSet

# GCP libraries
from google.cloud import storage

# Import organizational libraries
import time
import uuid
import os
import shutil
import pprint
import pdb
import json
import urllib.request
import subprocess

# Prep work
global email
hips = 'https://storage.googleapis.com/hips-data/images'
pp = pprint.PrettyPrinter(indent=2)
working_message = "Status updating..."
vendor_batch_id = 0
_HIPS_CUTOUTS = "hips_cutouts"
project_id = project.id
guid = ""
cutouts_dir = ""
progress_message = ""
manifest_url = ""
edc_response = ""
timestamp = None
before_zip = 0
after_zip = 0

def clean_up_unused_subject_set():
    global client
    global vendor_batch_id
    h.update("Cleaning up unused subject set on the Zooniverse platform, vendor_batch_id : " + str(vendor_batch_id))
    
    ss, etag = client.get(path="/subject_sets/" + str(vendor_batch_id))
    
    json_response = client.delete(path='/subject_sets/' + str(vendor_batch_id), headers={"If-Match":etag})
    
    return

def send_zooniverse_manifest():
    # import json
    global vendor_batch_id
    global manifest_url
    global client
    # subject_set.id
    # h.update("Sending project manifest to Zoonverse...")
    h.update("subject_set.id: " + str(vendor_batch_id) + "; manifest: " + manifest_url);

    payload = {"subject_set_imports": {"source_url": manifest_url, "links": {"subject_set": str(vendor_batch_id)}}}

    json_response, etag = client.post(path='/subject_set_imports', json=payload)
    return

def create_new_subject_set(name):
    h.update("Creating a new Zooniverse subject set")
    # Create a new subject set
    global project
    global panoptes_client
    global vendor_batch_id
    h.update(project.id)
    subject_set = panoptes_client.SubjectSet()
    subject_set.links.project = project

    # Give the subject set a display name (that will only be visible to you on the Zooniverse platform)
    subject_set.display_name = name 

    subject_set.save()
    project.reload()
    vendor_batch_id = subject_set.id
    return vendor_batch_id

def check_status():
    # global guid
    guid = "01b080e8-75b2-437d-ab7e-f8ec3be038a9"
    status_uri = "https://rsp-data-exporter-e3g4rcii3q-uc.a.run.app/citizen-science-ingest-status?guid=" + guid
    raw_response = urllib.request.urlopen(status_uri).read()
    response = raw_response.decode('UTF-8')
    return json.loads(response)


# Validates that the RSP user is allowed to create a new subject set
def send_data(subject_set_name, cutouts = None):
    h.update("Checking batch status")
    global manifest_url, edc_response
    if has_active_batch() == True:
        h.update("Active batch exists!!! Continuing because this notebook is in debug mode")
        # raise CitizenScienceError("You cannot send another batch of data while a subject set is still active on the Zooniverse platform - you can only send a new batch of data if all subject sets associated to a project have been completed.")
    if __cit_sci_data_type == _HIPS_CUTOUTS:
        zip_path = zip_hips_cutouts()
        upload_hips_cutouts(zip_path)
        subject_set_id = create_new_subject_set(subject_set_name)
        # if timestamp != None and ((round(time.time() * 1000)) - timestamp) > 300000:
        #     h.update("You must wait five minutes between sending batches of data, please try again in a few minutes.")
        #     clean_up_unused_subject_set()
        #     return # It has been less than 5 minutes since the user sent their last batch
        # else:
        #     timestamp = round(time.time() * 1000)
        
        edc_response = json.loads(alert_edc_of_new_citsci_data(subject_set_id))

    else:
        send_butler_data_to_edc()
        subject_set_id = create_new_subject_set(subject_set_name)
        manifest_url = send_butler_data_to_edc()
    
    if edc_response["status"] == "success":
        manifest_url = edc_response["manifest_url"]
        if len(edc_response["messages"]) > 0:
            h.update(edc_response["messages"])
        else:
            h.update(manifest_url)
    else:
        clean_up_unused_subject_set()
        # raise CitizenScienceError(edc_response["messages"])
        h.update(edc_response)
        return

    send_zooniverse_manifest()
    h.update("Transfer process complete, but further processing is required on the Zooniverse platform and you will receive an email at " + email)
    return

def zip_hips_cutouts():
    global before_zip, after_zip
    before_zip = round(time.time() * 1000)
    
    global guid
    guid = str(uuid.uuid4())
    cutouts_dir = "./project/citizen-science/astro-cutouts/"
    data_dir = cutouts_dir + guid
    os.mkdir(data_dir);
    
    h.update("Duplicating astro cutouts for testing purposes.")
    # beginning of temporary testing code
    for x in range(10005): # create 100 cutouts from the one cutout image
        plt.imsave(data_dir + "/cutout-" + str(round(time.time() * 1000)) + "-" + str(x) + ".png", result)
    # end of temporary testing code    
    
    #subprocess.check_output("zip_tool"
    h.update("Zipping up all the astro cutouts - this can take a few minutes with large data sets, but unlikely more than 10 minutes.")
    shutil.make_archive(cutouts_dir + guid, 'zip', data_dir)
    
    after_zip = round(time.time() * 1000)
    return [cutouts_dir + guid + '.zip', guid + '.zip']

def upload_hips_cutouts(zip_path):
    global before_zip, after_zip
    h.update("Uploading the citizen science data, zipping up took : ")
    bucket_name = "citizen-science-data"
    # service_account_key = "skyviewer-398f28c943e8.json" # replace this with the GCP key provided to you
    destination_blob_name = zip_path[1]
    source_file_name = zip_path[0]

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    return

def alert_edc_of_new_citsci_data(vendor_batch_id):
    project_id_str = str(project_id)
    h.update("Notifying the Rubin EPO Data Center of the new data, which will finish processing of the data and notify Zooniverse")
    # h.update("Vendor batch ID : " + str(vendor_batch_id))
    global guid
    
    try:
        edc_endpoint = "https://rsp-data-exporter-dot-skyviewer.uw.r.appspot.com/citizen-science-bucket-ingest?email=" + email + "&vendor_project_id=" + project_id_str + "&guid=" + guid + "&vendor_batch_id=" + str(vendor_batch_id) + "&debug=True"
        #edc_endpoint = "https://rsp-data-exporter-e3g4rcii3q-uc.a.run.app/citizen-science-bucket-ingest?email=" + email + "&vendor_project_id=" + project_id_str + "&guid=" + guid + "&vendor_batch_id=" + str(vendor_batch_id) + "&debug=True"
        response = urllib.request.urlopen(edc_endpoint).read()
        manifestUrl = response.decode('UTF-8')
        return manifestUrl
    except Exception as e:
        clean_up_unused_subject_set()
        h.update(e)
        return


def send_butler_data_to_edc():
    h.update("Notifying the Rubin EPO Data Center of the new data, which will finish processing of the data and notify Zooniverse")
    edcEndpoint = "https://rsp-data-exporter-e3g4rcii3q-uc.a.run.app/citizen-science-butler-ingest?email=" + email + "&collection=" + datasetId + "&sourceId=" + sourceId + "&vendorProjectId=" + str(projectId) + "&vendor_batch_id=" + str(vendor_batch_id)
    print('Processing data for Zooniverse, this may take up to a few minutes.')
    response = urllib.request.urlopen(edcEndpoint).read()
    manifestUrl = response.decode('UTF-8')
    return

def has_active_batch():
    active_batch = False
    for subject_set in project.links.subject_sets:
        for completeness_percent in list(subject_set.completeness.values()):
            if completeness_percent == 1.0:
                active_batch = True
                break
        if active_batch:
            break
    return active_batch

# Custom error handling for this notebook
class CitizenScienceError(Exception):
   
    # Constructor or Initializer
    def __init__(self, value):
        self.value = value
   
    # __str__ is to print() the value
    def __str__(self):
        return(repr(self.value))
    
print("Loaded Citizen Science SDK")
                            

In [ ]:
print(guid)